In [1]:
#Import required libraries and functions
from sahi import AutoDetectionModel
from sahi.utils.cv import read_image, read_image_as_pil
from sahi.utils.file import Path, increment_path, list_files, save_json, save_pickle, download_from_url
from sahi.predict import get_prediction, get_sliced_prediction, predict, agg_prediction, get_prediction_batched, get_sliced_prediction_batched, predict 
from sahi.prediction import visualize_object_predictions
from sahi.postprocess.utils import ObjectPredictionList, has_match, merge_object_prediction_pair
from IPython.display import Image
from numpy import asarray
from tqdm import tqdm
from PIL import Image
from sahi.prediction import ObjectPrediction, PredictionResult
from pathlib import Path
from multiprocessing import Pool, cpu_count
from torchvision.ops import nms, batched_nms, clip_boxes_to_image
import cv2
import math
import os
import time
import json
import numpy as np
import time
import torch

In [2]:
#create the model instance
pretrained_yolo_model_path = 'models/yolov8/last.pt'
#yolov8_model_path_baseline = 'models/baseline/yolov8n.pt'
detection_model = AutoDetectionModel.from_pretrained(
        model_type='yolov8',
        model_path=pretrained_yolo_model_path,
        confidence_threshold=0.3,
        device="cuda:0", # or 'cpu'
)

#### **Helper Functions**

In [3]:
#functions to set the overlap ratio and slice size as per detected objects count in each slice
def get_slice_parameters(object_density, slice_size):
    
    if object_density >= 50:
        #slice_size = min_dim // 4
        slice_width = slice_size
        slice_height = slice_size
        overlap_width_ratio = 0.5
        overlap_height_ratio = 0.5
    elif 25 <= object_density < 50:
        #slice_size = min_dim // 2
        slice_width = slice_size
        slice_height = slice_size
        overlap_width_ratio = 0.25
        overlap_height_ratio = 0.25
    elif 10 <= object_density < 25:
        #slice_size = min_dim // 2
        slice_width = slice_size
        slice_height = slice_size
        overlap_width_ratio = 0.15
        overlap_height_ratio = 0.15
    else:
        return

    return slice_width, slice_height, overlap_width_ratio, overlap_height_ratio

In [4]:
def calculate_box_density(b, B):
    """
    Calculate the density of box b based on nearby boxes in set B.

    Args:
    b: A box represented as [x1, x2, y1, y2] where:
       - (x1, y1) is bottom-left corner
       - (x2, y2) is top-right corner
    B: A list of boxes, each in the same format as b
    
    Returns:
        db: The density of box b (number of boxes in B within radius r of b's center)
    """
    db = 0  #density of box b
    x1, y1, x2, y2 = b
    xc = (x1 + x2) / 2
    yc = (y1 + y2) / 2
    r = min(x2 - x1, y2 - y1)
    
    for bi in B:
        xi1, yi1, xi2, yi2 = bi
        xic = (xi1 + xi2) / 2
        yic = (yi1 + yi2) / 2
        di = math.sqrt((xic - xc)**2 + (yic - yc)**2)
        if di <= r / 2:
            db += 1
    
    return db

In [5]:
def calculate_iou(box1, box2):
    """
    Calculate Intersection over Union (IoU) between two boxes.
    
    Args:
        box1, box2: Boxes in format [x1, y1, x2, y2]
    
    Returns:
        IoU value between 0 and 1
    """
    x1_1, y1_1, x2_1, y2_1 = box1
    x1_2, y1_2, x2_2, y2_2 = box2
    
    # Intersection coordinates
    x1_inter = max(x1_1, x1_2)
    y1_inter = max(y1_1, y1_2)
    x2_inter = min(x2_1, x2_2)
    y2_inter = min(y2_1, y2_2)
    
    # Intersection area
    inter_width = max(0, x2_inter - x1_inter)
    inter_height = max(0, y2_inter - y1_inter)
    intersection_area = inter_width * inter_height
    
    # Areas of each box
    area1 = (x2_1 - x1_1) * (y2_1 - y1_1)
    area2 = (x2_2 - x1_2) * (y2_2 - y1_2)
    
    # Union area
    union_area = area1 + area2 - intersection_area
    
    return intersection_area / union_area if union_area > 0 else 0.0


In [7]:
def nms_with_pbde_filter(S, B, Nt, Np, score_threshold=0.3, cluster_size_min=1):
    """
    Non-Maximum Suppression with Probabilistic Box Density Estimation,
    enhanced to preserve true positives in sparse regions via threshold-guarded isolated-box bypass.

    Args:
        S: List or array of detection scores corresponding to boxes in B
        B: List or array of detection boxes in format [x1, x2, y1, y2]
        Nt: NMS IoU threshold
        Np: PBDE density threshold
        score_threshold: confidence threshold above which boxes are always kept
        cluster_size_min: max number of overlapping neighbors (IoU>=Nt) to treat box as isolated

    Returns:
        D: List of selected boxes
        selected_scores: List of scores for selected boxes
    """
    D = []
    selected_scores = []
    print("PBDE Threshold:", Np)
    print("Confidence Score threshold:", score_threshold)

    # Working copies
    B_work = B.tolist().copy()
    S_work = S.tolist().copy()

    while B_work:
        # 1) Pick highest-score box
        m = S_work.index(max(S_work))
        M = B_work.pop(m)
        max_score = S_work.pop(m)

        print("Max Score: ", max_score)
        # 2) High-confidence bypass
        if max_score >= score_threshold:
            #print("TRUE SCORE")
            # Remove overlaps to avoid duplicates
            for i in reversed(range(len(B_work))):
                if calculate_iou(M, B_work[i]) >= Nt:
                    B_work.pop(i)
                    S_work.pop(i)
            D.append(M)
            selected_scores.append(max_score)
            continue

        # 3) Find overlapping neighbors
        neighbors = [i for i, bi in enumerate(B_work) if calculate_iou(M, bi) >= Nt]

        print("---------------------------------")
        print("Neighbour Count: ", len(neighbors))
        
        # 4) Thresholded isolated-box bypass (sparse-region)
        if len(neighbors) <= cluster_size_min:
            print("true")
            # Only keep isolated boxes if their score meets threshold
            if max_score >= score_threshold:
                # Remove any small overlaps
                for i in reversed(neighbors):
                    B_work.pop(i)
                    S_work.pop(i)
                D.append(M)
                selected_scores.append(max_score)
            # Otherwise discard low-score isolated box
            continue

        # 5) Compute local density and apply standard PBDE
        c = calculate_box_density(M, B_work)
        print("Calculated box density:", c)

        # Standard NMS overlap removal
        for i in reversed(range(len(B_work))):
            if calculate_iou(M, B_work[i]) >= Nt:
                B_work.pop(i)
                S_work.pop(i)

        # Only keep if density exceeds threshold
        if c > Np:
            D.append(M)
            selected_scores.append(max_score)
    
    print("^^^^^^^^^^^^^^^^^^")        
    print("Selected Boxes Count (inside filter): ", len(D))
    return D, selected_scores


In [8]:
# Apply Non-Maximum Suppression (NMS) with PBDE
def apply_merge(detections, iou_threshold=0.5, pbde_threshold=2):
    boxes, scores, labels = [], [], []
    for det in detections:
        boxes.append(det.bbox.to_xyxy())
        scores.append(det.score.value)
        labels.append(det.category.id)
    boxes = torch.tensor(boxes, dtype=torch.float32)
    scores = torch.tensor(scores, dtype=torch.float32)
    labels = torch.tensor(labels)
    count = 0
    final_detections = []
    unique_labels = labels.unique()
    for label in unique_labels:
        idxs = (labels == label).nonzero(as_tuple=False).squeeze(1)
        class_boxes = boxes[idxs]
        class_scores = scores[idxs]
 
        selected_boxes, selected_scores = nms_with_pbde_filter(class_scores, class_boxes, iou_threshold, pbde_threshold)

        print("+++++++++++++++")
        print("Filtered Boxes (Outside filter): ", len(selected_boxes))
        

        # Find original detection indices for selected boxes
        for i, selected_box in enumerate(selected_boxes):
            for j, idx in enumerate(idxs):
                #print("Idx:", idx)
                
                # convert list -> tensor to allow simple exact comparison
                sb = selected_box if isinstance(selected_box, torch.Tensor) \
                     else torch.as_tensor(selected_box, dtype=class_boxes.dtype, device=class_boxes.device)
                sb = sb.flatten()  # ensure shape (4,)
                
                # simple exact tensor comparison against the class subset
                if torch.equal(class_boxes[j], sb):
                    detection = detections[int(idx.item())]

                    # use the selected (filtered) box coords
                    bbox_xyxy = sb.tolist()
                    detection.bbox.x1 = float(bbox_xyxy[0])
                    detection.bbox.y1 = float(bbox_xyxy[1])
                    detection.bbox.x2 = float(bbox_xyxy[2])
                    detection.bbox.y2 = float(bbox_xyxy[3])

                    # update confidence score
                    detection.score.value = float(
                        selected_scores[i].item() if hasattr(selected_scores[i], "item") else selected_scores[i]
                    )

                    final_detections.append(detection)
                    count = count + 1
                    print("Count:", count)
                    break

    print("Final Detection Count: ", len(final_detections))
    return final_detections

In [9]:
#convert final predictions after NMS merge to COCO format
def merged_preds_to_coco(preds, image_id):
    coco_predictions = []
    for pred in preds:
        try:
            coco = pred.to_coco_prediction(image_id=image_id)
            coco_dict = {
                "image_id": coco.image_id,
                "bbox": coco.bbox,
                "score": coco.score,
                "category_id": coco.category_id,
                "segmentation": coco.segmentation,
                "iscrowd": coco.iscrowd,
                "area": coco.area,
            }
            # Optionally include category_name (not used in COCO eval)
            # coco_dict["category_name"] = coco.category_name

            # Only add if bbox is valid (non-empty)
            if coco_dict["bbox"]:
                coco_predictions.append(coco_dict)
        except Exception as e:
            print(f"Failed to convert to COCO dict: {e}")
    return coco_predictions

In [13]:
# Function to get image details by image_id
def get_image_id(coco_data, image_name):
    for image in coco_data["images"]:
        file_name = Path(image['file_name']).stem
        if file_name == image_name:
            return image['id']
    return None

In [11]:
def predict_multilevel_fine_sliced_images(input_folder, dataset_json_path, detection_model, base_slice_size=512):

    print("\n***********************************************")
    print("Adaptive MultiLevel SAHI Prediction")
    print("*************************************************")
    
    name = "exp"
    save_dir = Path(increment_path(Path("sliced_predictions") / name, exist_ok=False))
    os.makedirs(save_dir, exist_ok=True)

    if dataset_json_path:
        with open(dataset_json_path, "r") as file:
            data = json.load(file)
        
    vis_params = {
        "bbox_thickness": 2,
        "text_size": 0.5,
        "text_thickness": 1,
        "hide_labels": False,
        "hide_conf": False,
        "format": "png"
    }

    image_files = [
        f for f in os.listdir(input_folder)
        if f.lower().endswith((".jpg", ".jpeg", ".png"))
    ]

    print(f"\nRunning fine slicing prediction on {len(image_files)} image(s)...")
    total_prediction_time = 0.0
    grand_total_predictions = 0
    all_coco_preds = []

    for filename in image_files:
        image_path = os.path.join(input_folder, filename)
        image_pil = Image.open(image_path).convert("RGB")
        image_np = np.array(image_pil)
        image_h, image_w = image_np.shape[:2]
        filename_wo_ext = Path(filename).stem
        total_prediction_count = 0
        print("*****************************************")
        print("File Name: ", filename_wo_ext)
        print("*****************************************")
        
        img_id = get_image_id(data, filename_wo_ext) if dataset_json_path else None
        all_object_predictions = []

        #Base prediction
        t0 = time.time()
        intial_prediction = get_prediction(image_path, detection_model)
        t_base = (time.time() - t0)
        print(f"Base pred time: {t_base * 1000:.2f} ms")
        total_prediction_time += t_base
        print(f"Total Based pred time: {total_prediction_time * 1000:.2f} ms")
        full_preds = intial_prediction.object_prediction_list
        full_object_density = len(full_preds)
        print("Full Image Object Density:", full_object_density)

        for pred in full_preds:
            all_object_predictions.append(pred)
            
        total_prediction_count += len(all_object_predictions)
    
        # Split image into 2x2 grid
        grid_h, grid_w = image_h // 2, image_w // 2

        for row in range(2):
            for col in range(2):
                x1, y1 = col * grid_w, row * grid_h
                x2, y2 = min(x1 + grid_w, image_w), min(y1 + grid_h, image_h)
                sub_img = image_pil.crop((x1, y1, x2, y2))
                print("Cropped Image Dimension:", x1, y1, x2, y2)

                time_start = time.time()
                base_pred = get_prediction(sub_img, detection_model)
                time_end = time.time() - time_start
                
                print("Initial Prediction time is: {:.2f} ms".format(time_end * 1000))

                coarse_preds = base_pred.object_prediction_list
                object_density = len(coarse_preds)
                print("Slice Object Density:", object_density)
                
                slice_params = get_slice_parameters(object_density, base_slice_size)
            
                iteration_time = time_end
                
                preds = coarse_preds.copy()
                
                if slice_params:
                    slice_width, slice_height, overlap_w, overlap_h = slice_params
                    print("********* Slice Parameters ***********")
                    print("Slice Width: ", slice_width)
                    print("Slice Height: ", slice_height)
                    print("Overlap Width Ratio: ", overlap_w)
                    print("Overlap Height Ratio: ", overlap_h)

                    time_start_slice = time.time()
                    sliced_pred = get_sliced_prediction(
                        sub_img,
                        detection_model,
                        slice_height=slice_height,
                        slice_width=slice_width,
                        overlap_height_ratio=overlap_h,
                        overlap_width_ratio=overlap_w,
                        postprocess_type="NMS",
                        postprocess_match_metric="IOU",
                        postprocess_match_threshold=0.3,
                        postprocess_min_area=16,
                        verbose=0
                    )
                    time_end_slice = time.time() - time_start_slice
                    print("Sliced Prediction time is: {:.2f} ms".format(time_end_slice * 1000))
                    
                    iteration_time += time_end_slice
                    fine_preds = sliced_pred.object_prediction_list
                    print("Pre Adaptive Slicing: ", len(preds))
                    preds.extend(fine_preds)
                    print("Post Adaptive Slicing: ", len(preds))
                else:
                    print("Prediction time is: {:.2f} ms".format(time_end * 1000))

                total_prediction_time += iteration_time
                
                #Offset bounding boxes back to original image space
                for pred in preds:
                    pred.bbox.minx += x1
                    pred.bbox.maxx += x1
                    pred.bbox.miny += y1
                    pred.bbox.maxy += y1
                    all_object_predictions.append(pred)
        
                total_prediction_count += len(preds)

        print("________________________________________________")

        print("Total Prediction Count (Crop+Slice): ", len(all_object_predictions))
        
        # Merge all predictions for image
        merged_preds = apply_merge(all_object_predictions, iou_threshold=0.5, pbde_threshold=1)
        print("Total Prediction Count (Merge): ", len(merged_preds))
        
        #object_prediction_list = ObjectPredictionList(all_object_predictions)
        #merged_keep = truncated_nms_merge(object_prediction_list.totensor())
        #selected_object_predictions = object_prediction_list[merged_keep].tolist()
        #if not isinstance(selected_object_predictions, list):
        #    selected_object_predictions = [selected_object_predictions]
        
        # Convert merged predictions to COCO format
        if dataset_json_path:
            coco_preds = merged_preds_to_coco(merged_preds, img_id)
            all_coco_preds.extend(coco_preds)
            
        # Visualization
        visualize_object_predictions(
            image=np.ascontiguousarray(image_pil),
            object_prediction_list=merged_preds,
            rect_th=vis_params["bbox_thickness"],
            text_size=vis_params["text_size"],
            text_th=vis_params["text_thickness"],
            hide_labels=vis_params["hide_labels"],
            hide_conf=vis_params["hide_conf"],
            output_dir=save_dir,
            file_name=filename_wo_ext,
            export_format=vis_params["format"]
        )

        grand_total_predictions += total_prediction_count

    if dataset_json_path:
        save_path = str(save_dir / "result.json")
        save_json(all_coco_preds, save_path)
   
    print(f"\nCompleted {len(image_files)} images.")
    print("Total Prediction Count (Before NMS Merge): ", grand_total_predictions)
    print("Total Prediction Count (After Final NMS Merge): ", len(merged_preds))
    print("Total Prediction time for all images is: {:.2f} ms".format(total_prediction_time * 1000))
    print(f"Prediction results are successfully exported to {save_dir}")


In [22]:
# Example usage:# Example usage: Adaptive-Optimized-NMS Take 2(Adaptive-OptNMS)  --> best pbde-2; conf:0.3
source_folder = './single_test/images'
json_path = "./subset_visdrone_test_990.json"
slice_size = 256

# Run fine-sliced adaptive prediction
result = predict_multilevel_fine_sliced_images(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              
)


***********************************************
Adaptive MultiLevel SAHI Prediction
*************************************************

Running fine slicing prediction on 1 image(s)...
*****************************************
File Name:  0000006_06773_d_0000018
*****************************************
Base pred time: 98.78 ms
Total Based pred time: 98.78 ms
Full Image Object Density: 37
Cropped Image Dimension: 0 0 680 382
Initial Prediction time is: 8.48 ms
Slice Object Density: 15
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 6
POST PROCESS:  NMS
Confidence Score:  0.3
Original Prediction Count 39
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  15
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  3
Final Bounding Box Count (NMS):  1
Sliced Prediction time is: 64.36 ms
Pre Adaptive Slicing:  15
Post Adaptive Slicing:  36
Cropped Image Dimensi

In [23]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_990.json' --result_json_path './sliced_predictions/exp1012/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.01s).
Accumulating evaluation results...
DONE (t=0.00s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.454
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.810
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.532
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.976
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.826
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.574
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.435
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.523
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [11]:
# Example usage:# Example usage: Adaptive-Optimized-NMS Take 2(Adaptive-OptNMS)
source_folder = './test_subsets/v5/images'
json_path = "./subset_visdrone_test_data_15_v5.json"
slice_size = 256

# Run fine-sliced adaptive prediction
result = predict_multilevel_fine_sliced_images(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              
)


***********************************************
Adaptive MultiLevel SAHI Prediction
*************************************************

Running fine slicing prediction on 15 image(s)...
*****************************************
File Name:  9999963_00000_d_0000057
*****************************************


/mmfs1/cm/shared/apps_local/python/3.11/envs/torch11.8/lib/python3.9/site-packages/torch/nn/modules/conv.py:456: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv2d(input, weight, bias, self.stride,


Base pred time: 671.79 ms
Total Based pred time: 671.79 ms
Full Image Object Density: 33
Cropped Image Dimension: 0 0 700 525
Initial Prediction time is: 9.49 ms
Slice Object Density: 3
Prediction time is: 9.49 ms
Cropped Image Dimension: 700 0 1400 525
Initial Prediction time is: 9.11 ms
Slice Object Density: 20
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 12
POST PROCESS:  NMS
Original Prediction Count 31
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  18
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  1
Sliced Prediction time is: 112.48 ms
Pre Adaptive Slicing:  20
Post Adaptive Slicing:  42
Cropped Image Dimension: 0 525 700 1050
Initial Prediction time is: 8.59 ms
Slice Object Density: 7
Prediction time is: 8.59 ms
Cropped Image Dimension: 700 525 1400 1050
Initial Prediction time is: 8.46 ms
Slice Obje

In [12]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_15_v5.json' --result_json_path './sliced_predictions/exp980/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.09s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.108
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.170
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.117
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.110
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.207
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.512
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.064
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.127
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [14]:
# Example usage:# Example usage: Adaptive-Optimized-NMS Take 2(Adaptive-OptNMS) --> PBDE -1 confidence score: 0.3
source_folder = './test_subsets/v5/images'
json_path = "./subset_visdrone_test_data_15_v5.json"
slice_size = 256

# Run fine-sliced adaptive prediction
result = predict_multilevel_fine_sliced_images(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              
)


***********************************************
Adaptive MultiLevel SAHI Prediction
*************************************************

Running fine slicing prediction on 15 image(s)...
*****************************************
File Name:  9999963_00000_d_0000057
*****************************************


/mmfs1/cm/shared/apps_local/python/3.11/envs/torch11.8/lib/python3.9/site-packages/torch/nn/modules/conv.py:456: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv2d(input, weight, bias, self.stride,


Base pred time: 552.58 ms
Total Based pred time: 552.58 ms
Full Image Object Density: 33
Cropped Image Dimension: 0 0 700 525
Initial Prediction time is: 9.41 ms
Slice Object Density: 3
Prediction time is: 9.41 ms
Cropped Image Dimension: 700 0 1400 525
Initial Prediction time is: 9.01 ms
Slice Object Density: 20
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 12
POST PROCESS:  NMS
Confidence Score:  0.3
Original Prediction Count 31
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  18
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  1
Sliced Prediction time is: 110.59 ms
Pre Adaptive Slicing:  20
Post Adaptive Slicing:  42
Cropped Image Dimension: 0 525 700 1050
Initial Prediction time is: 8.38 ms
Slice Object Density: 7
Prediction time is: 8.38 ms
Cropped Image Dimension: 700 525 1400 1050
Initial Prediction time

In [15]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_15_v5.json' --result_json_path './sliced_predictions/exp1033/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.22s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.165
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.266
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.177
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.170
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.304
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.613
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.089
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.189
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [36]:
# Example usage:# Example usage: Adaptive-Optimized-NMS Take 2(Adaptive-OptNMS) --> PBDE -2 confidence score: 0.3
source_folder = './test_subsets/v5/images'
json_path = "./subset_visdrone_test_data_15_v5.json"
slice_size = 256

# Run fine-sliced adaptive prediction
result = predict_multilevel_fine_sliced_images(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              
)


***********************************************
Adaptive MultiLevel SAHI Prediction
*************************************************

Running fine slicing prediction on 15 image(s)...
*****************************************
File Name:  9999963_00000_d_0000057
*****************************************
Base pred time: 34.09 ms
Total Based pred time: 34.09 ms
Full Image Object Density: 33
Cropped Image Dimension: 0 0 700 525
Initial Prediction time is: 9.75 ms
Slice Object Density: 3
Prediction time is: 9.75 ms
Cropped Image Dimension: 700 0 1400 525
Initial Prediction time is: 9.48 ms
Slice Object Density: 20
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 12
POST PROCESS:  NMS
Confidence Score:  0.3
Original Prediction Count 31
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  18
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  1
Final Bounding 

In [37]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_15_v5.json' --result_json_path './sliced_predictions/exp992/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.22s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.165
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.266
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.177
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.170
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.304
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.613
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.089
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.189
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [11]:
# Example usage:# Example usage: Adaptive-Optimized-NMS Take 2(Adaptive-OptNMS)  --> lower conf higher PBDE
source_folder = './test_subsets/v5/images'
json_path = "./subset_visdrone_test_data_15_v5.json"
slice_size = 256

# Run fine-sliced adaptive prediction
result = predict_multilevel_fine_sliced_images(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              
)


***********************************************
Adaptive MultiLevel SAHI Prediction
*************************************************

Running fine slicing prediction on 15 image(s)...
*****************************************
File Name:  9999963_00000_d_0000057
*****************************************


/mmfs1/cm/shared/apps_local/python/3.11/envs/torch11.8/lib/python3.9/site-packages/torch/nn/modules/conv.py:456: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv2d(input, weight, bias, self.stride,


Base pred time: 5879.07 ms
Total Based pred time: 5879.07 ms
Full Image Object Density: 33
Cropped Image Dimension: 0 0 700 525
Initial Prediction time is: 9.54 ms
Slice Object Density: 3
Prediction time is: 9.54 ms
Cropped Image Dimension: 700 0 1400 525
Initial Prediction time is: 9.10 ms
Slice Object Density: 20
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 12
POST PROCESS:  NMS
Original Prediction Count 31
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  18
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  1
Sliced Prediction time is: 462.44 ms
Pre Adaptive Slicing:  20
Post Adaptive Slicing:  42
Cropped Image Dimension: 0 525 700 1050
Initial Prediction time is: 8.67 ms
Slice Object Density: 7
Prediction time is: 8.67 ms
Cropped Image Dimension: 700 525 1400 1050
Initial Prediction time is: 8.51 ms
Slice Ob

In [12]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_15_v5.json' --result_json_path './sliced_predictions/exp981/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.26s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.165
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.266
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.177
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.170
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.304
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.613
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.089
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.189
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [17]:
# Example usage:# Example usage: Adaptive-Optimized-NMS Take 2(Adaptive-OptNMS) --> PBDE -3
source_folder = './test_subsets/v5/images'
json_path = "./subset_visdrone_test_data_15_v5.json"
slice_size = 256

# Run fine-sliced adaptive prediction
result = predict_multilevel_fine_sliced_images(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              
)


***********************************************
Adaptive MultiLevel SAHI Prediction
*************************************************

Running fine slicing prediction on 15 image(s)...
*****************************************
File Name:  9999963_00000_d_0000057
*****************************************
Base pred time: 32.80 ms
Total Based pred time: 32.80 ms
Full Image Object Density: 33
Cropped Image Dimension: 0 0 700 525
Initial Prediction time is: 9.18 ms
Slice Object Density: 3
Prediction time is: 9.18 ms
Cropped Image Dimension: 700 0 1400 525
Initial Prediction time is: 9.06 ms
Slice Object Density: 20
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 12
POST PROCESS:  NMS
Original Prediction Count 31
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  18
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  1
Sli

In [18]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_15_v5.json' --result_json_path './sliced_predictions/exp982/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.22s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.165
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.266
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.177
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.170
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.304
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.613
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.089
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.189
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [20]:
# Example usage:# Example usage: Adaptive-Optimized-NMS Take 2(Adaptive-OptNMS) --> PBDE -6
source_folder = './test_subsets/v5/images'
json_path = "./subset_visdrone_test_data_15_v5.json"
slice_size = 256

# Run fine-sliced adaptive prediction
result = predict_multilevel_fine_sliced_images(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              
)


***********************************************
Adaptive MultiLevel SAHI Prediction
*************************************************

Running fine slicing prediction on 15 image(s)...
*****************************************
File Name:  9999963_00000_d_0000057
*****************************************
Base pred time: 33.30 ms
Total Based pred time: 33.30 ms
Full Image Object Density: 33
Cropped Image Dimension: 0 0 700 525
Initial Prediction time is: 9.33 ms
Slice Object Density: 3
Prediction time is: 9.33 ms
Cropped Image Dimension: 700 0 1400 525
Initial Prediction time is: 11.06 ms
Slice Object Density: 20
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 12
POST PROCESS:  NMS
Original Prediction Count 31
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  18
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  1
Sl

In [21]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_15_v5.json' --result_json_path './sliced_predictions/exp983/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.22s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.165
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.266
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.177
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.170
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.304
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.613
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.089
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.189
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [18]:
# Example usage:# Example usage: Adaptive-Optimized-NMS Take 2(Adaptive-OptNMS) --> PBDE -8
source_folder = './test_subsets/v5/images'
json_path = "./subset_visdrone_test_data_15_v5.json"
slice_size = 256

# Run fine-sliced adaptive prediction
result = predict_multilevel_fine_sliced_images(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              
)


***********************************************
Adaptive MultiLevel SAHI Prediction
*************************************************

Running fine slicing prediction on 15 image(s)...
*****************************************
File Name:  9999963_00000_d_0000057
*****************************************
Base pred time: 32.82 ms
Total Based pred time: 32.82 ms
Full Image Object Density: 33
Cropped Image Dimension: 0 0 700 525
Initial Prediction time is: 9.55 ms
Slice Object Density: 3
Prediction time is: 9.55 ms
Cropped Image Dimension: 700 0 1400 525
Initial Prediction time is: 9.32 ms
Slice Object Density: 20
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 12
POST PROCESS:  NMS
Confidence Score:  0.3
Original Prediction Count 31
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  18
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  1
Final Bounding 

In [19]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_15_v5.json' --result_json_path './sliced_predictions/exp986/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.22s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.165
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.266
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.177
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.170
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.304
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.613
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.089
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.189
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [21]:
# Example usage:# Example usage: Adaptive-Optimized-NMS Take 2(Adaptive-OptNMS) --> PBDE -8
source_folder = './test_subsets/v5/images'
json_path = "./subset_visdrone_test_data_15_v5.json"
slice_size = 256

# Run fine-sliced adaptive prediction
result = predict_multilevel_fine_sliced_images(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              
)


***********************************************
Adaptive MultiLevel SAHI Prediction
*************************************************

Running fine slicing prediction on 15 image(s)...
*****************************************
File Name:  9999963_00000_d_0000057
*****************************************
Base pred time: 34.62 ms
Total Based pred time: 34.62 ms
Full Image Object Density: 33
Cropped Image Dimension: 0 0 700 525
Initial Prediction time is: 9.86 ms
Slice Object Density: 3
Prediction time is: 9.86 ms
Cropped Image Dimension: 700 0 1400 525
Initial Prediction time is: 9.64 ms
Slice Object Density: 20
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 12
POST PROCESS:  NMS
Confidence Score:  0.3
Original Prediction Count 31
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  18
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  1
Final Bounding 

In [24]:
# Example usage:# Example usage: Adaptive-Optimized-NMS Take 2(Adaptive-OptNMS) --> PBDE -6 confidence score: 0.6
source_folder = './test_subsets/v5/images'
json_path = "./subset_visdrone_test_data_15_v5.json"
slice_size = 256

# Run fine-sliced adaptive prediction
result = predict_multilevel_fine_sliced_images(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              
)


***********************************************
Adaptive MultiLevel SAHI Prediction
*************************************************

Running fine slicing prediction on 15 image(s)...
*****************************************
File Name:  9999963_00000_d_0000057
*****************************************
Base pred time: 29.90 ms
Total Based pred time: 29.90 ms
Full Image Object Density: 33
Cropped Image Dimension: 0 0 700 525
Initial Prediction time is: 9.66 ms
Slice Object Density: 3
Prediction time is: 9.66 ms
Cropped Image Dimension: 700 0 1400 525
Initial Prediction time is: 9.42 ms
Slice Object Density: 20
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 12
POST PROCESS:  NMS
Confidence Score:  0.3
Original Prediction Count 31
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  18
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  1
Final Bounding 

In [25]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_15_v5.json' --result_json_path './sliced_predictions/exp988/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.08s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.107
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.167
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.116
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.107
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.207
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.512
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.063
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.127
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [27]:
# Example usage:# Example usage: Adaptive-Optimized-NMS Take 2(Adaptive-OptNMS) --> PBDE -3 confidence score: 0.6
source_folder = './test_subsets/v5/images'
json_path = "./subset_visdrone_test_data_15_v5.json"
slice_size = 256

# Run fine-sliced adaptive prediction
result = predict_multilevel_fine_sliced_images(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              
)


***********************************************
Adaptive MultiLevel SAHI Prediction
*************************************************

Running fine slicing prediction on 15 image(s)...
*****************************************
File Name:  9999963_00000_d_0000057
*****************************************
Base pred time: 34.83 ms
Total Based pred time: 34.83 ms
Full Image Object Density: 33
Cropped Image Dimension: 0 0 700 525
Initial Prediction time is: 9.85 ms
Slice Object Density: 3
Prediction time is: 9.85 ms
Cropped Image Dimension: 700 0 1400 525
Initial Prediction time is: 9.37 ms
Slice Object Density: 20
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 12
POST PROCESS:  NMS
Confidence Score:  0.3
Original Prediction Count 31
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  18
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  1
Final Bounding 

In [28]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_15_v5.json' --result_json_path './sliced_predictions/exp989/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.08s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.107
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.167
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.116
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.107
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.207
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.512
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.063
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.127
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [30]:
# Example usage:# Example usage: Adaptive-Optimized-NMS Take 2(Adaptive-OptNMS) --> PBDE -1 confidence score: 0.6
source_folder = './test_subsets/v5/images'
json_path = "./subset_visdrone_test_data_15_v5.json"
slice_size = 256

# Run fine-sliced adaptive prediction
result = predict_multilevel_fine_sliced_images(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              
)


***********************************************
Adaptive MultiLevel SAHI Prediction
*************************************************

Running fine slicing prediction on 15 image(s)...
*****************************************
File Name:  9999963_00000_d_0000057
*****************************************
Base pred time: 34.14 ms
Total Based pred time: 34.14 ms
Full Image Object Density: 33
Cropped Image Dimension: 0 0 700 525
Initial Prediction time is: 9.97 ms
Slice Object Density: 3
Prediction time is: 9.97 ms
Cropped Image Dimension: 700 0 1400 525
Initial Prediction time is: 9.61 ms
Slice Object Density: 20
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 12
POST PROCESS:  NMS
Confidence Score:  0.3
Original Prediction Count 31
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  18
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  1
Final Bounding 

In [31]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_15_v5.json' --result_json_path './sliced_predictions/exp990/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.10s).
Accumulating evaluation results...
DONE (t=0.05s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.138
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.213
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.152
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.120
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.245
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.574
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.067
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.156
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [33]:
# Example usage:# Example usage: Adaptive-Optimized-NMS Take 2(Adaptive-OptNMS) --> PBDE -2 confidence score: 0.6
source_folder = './test_subsets/v5/images'
json_path = "./subset_visdrone_test_data_15_v5.json"
slice_size = 256

# Run fine-sliced adaptive prediction
result = predict_multilevel_fine_sliced_images(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              
)


***********************************************
Adaptive MultiLevel SAHI Prediction
*************************************************

Running fine slicing prediction on 15 image(s)...
*****************************************
File Name:  9999963_00000_d_0000057
*****************************************
Base pred time: 34.43 ms
Total Based pred time: 34.43 ms
Full Image Object Density: 33
Cropped Image Dimension: 0 0 700 525
Initial Prediction time is: 9.86 ms
Slice Object Density: 3
Prediction time is: 9.86 ms
Cropped Image Dimension: 700 0 1400 525
Initial Prediction time is: 9.33 ms
Slice Object Density: 20
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 12
POST PROCESS:  NMS
Confidence Score:  0.3
Original Prediction Count 31
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  18
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  1
Final Bounding 

In [34]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_15_v5.json' --result_json_path './sliced_predictions/exp991/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.09s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.108
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.170
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.117
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.110
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.207
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.512
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.064
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.127
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [41]:
# Example usage:# Example usage: Adaptive-Optimized-NMS Take 2(baseline model) --> PBDE -2 confidence score: 0.6
source_folder = './test_subsets/v5/images'
json_path = "./subset_visdrone_test_data_15_v5.json"
slice_size = 256

# Run fine-sliced adaptive prediction
result = predict_multilevel_fine_sliced_images(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              
)


***********************************************
Adaptive MultiLevel SAHI Prediction
*************************************************

Running fine slicing prediction on 15 image(s)...
*****************************************
File Name:  9999963_00000_d_0000057
*****************************************
Base pred time: 135.47 ms
Total Based pred time: 135.47 ms
Full Image Object Density: 7
Cropped Image Dimension: 0 0 700 525
Initial Prediction time is: 10.98 ms
Slice Object Density: 2
Prediction time is: 10.98 ms
Cropped Image Dimension: 700 0 1400 525
Initial Prediction time is: 10.59 ms
Slice Object Density: 10
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 12
POST PROCESS:  NMS
Confidence Score:  0.3
Original Prediction Count 24
Final Bounding Box Count (NMS):  12
Final Bounding Box Count (NMS):  1
Sliced Prediction time is: 134.15 ms
Pre Adaptive Slicing:  10
Post Adaptive Slic

In [42]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_15_v5.json' --result_json_path './sliced_predictions/exp994/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.02s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.003
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.005
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.003
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.004
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.016
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.002
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.008
 Average Precision  (AP) @[ IoU=0.50:0.95 | a